In [2]:
import requests
import os
import sys
import pandas as pd
import numpy as np

# import key sklearn ml algos
# import key sklearn metrics
# import 

from typing import Optional
import glob
# Add the parent directory of this notebook to sys.path
notebook_dir = os.path.dirname(os.path.abspath('__file__'))
parent_dir = os.path.dirname(notebook_dir)
sys.path.append(parent_dir)
from project_tools import project_utils, project_class

import datetime
import json
from tqdm import tqdm
from tqdm.notebook import tqdm
import gc
# import ds_utils
import random
import matplotlib.pyplot as plt
%matplotlib inline
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)

In [ ]:
df['feature_missing'] = df['feature'].isnull().astype(int)
df['feature'].fillna(df['feature'].mean(), inplace=True)

In [ ]:
def convert_timestamp_to_seconds(timestamp: str) -> float:
    """
    Convert text timestamp in format "M.SS.ss" to total seconds
    where M=minutes, SS=seconds, ss=decimal seconds
    
    Examples:
        "1.41.91" -> 101.91 (1 min 41.91 sec)
        "1.40.12" -> 100.12 (1 min 40.12 sec)
        "0.58.41" -> 58.41 (58.41 sec)
    
    Args:
        timestamp: String timestamp in M.SS.ss format
        
    Returns:
        Float value representing total seconds
    """
    parts = timestamp.split('.')
    
    if len(parts) != 3:
        raise ValueError(f"Invalid timestamp format: {timestamp}. Expected format: M.SS.ss")
        
    minutes = float(parts[0])
    seconds = float(parts[1])
    decimal = float(parts[2]) / 100  # Convert decimal part to fraction
    
    total_seconds = minutes * 60 + seconds + decimal
    
    return total_seconds


In [ ]:
def convert_date_to_int(date_str: str) -> int:
    """
    Convert date string in YYYY-MM-DD format to integer that preserves ordering
    
    Args:
        date_str: Date string in YYYY-MM-DD format
        
    Returns:
        Integer in format YYYYMMDD
        
    Example:
        '2015-11-18' -> 20151118b
        '2016-03-31' -> 20160331
    """
    # Remove hyphens and convert to integer
    return int(date_str.replace('-', ''))

# Example usage:
dates = ['2015-11-18', '2015-03-25', '2016-03-31', '2015-07-05', '2016-11-06']
date_ints = [convert_date_to_int(d) for d in dates]
print(f"Original dates: {dates}")
print(f"Integer dates: {date_ints}")


In [ ]:
# running position calculation instructions 
# given a horse race dataframe, each row provide information about a horse in a specific race,  with the following relevant columns:
#  - horse_id - id of a horse
# - is_winer - if the horse is a winer 
# - is_top3 - if the horse finished in top 3

# the dataframe is sorted by time order with records of earlier race on top
# write a function that calculate, and return the following for each horse at each race:
# - running average of position of the last 3 races
# - running average of position of the last 5 races
# - running average of position of the last 7 races

# for each results above, output a column

# specific instruction:
# - if at any given race, for a given horse, there isn't enough races to make up the running number of 3, 5 or 7, return NaN
# - for any position with 99 - treat this as missing value 




function instruction to calculate horse prediction metrics
with a horse racing dataframe, each row provide information about a horse in a specific race,  with the following relevant columns:
- horse_id - id of a horse
- is_winer - if the horse is a winer - 0 or 1
- is_top3 - if the horse finished in top 3 - 0 or 1
- race_id - id of a race
- draw number - the draw number of the horse in the race

now, given a list of races, for each race, given a list of predictions for each of the horse winning the race
- ordered by the draw number, 
- the higher the value the more likely the horse is to win

propose what is the best metric to evaluate the prediction quality of prediction top 1 and top 3 for a list of races



do the following for each race:
- convert the list of predictions into rank
- check if the rank 1 (lowest value) prediction is the same as the "is_winner" of the horse in the race
- check if the top rank 3 prediction is the same as the "is_top3" of the horse in the race
- some race might have more than 3 top_3, i.e. more than 3 is_top3 = 1, in this case, check if the top 3 predictions contains label "1"

return two dictionaries:
- one for top 1 prediction:



